# Linear Regrssion Model 6 - Transform Target Column 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from   sklearn.impute        import *
from sklearn.linear_model    import *
from sklearn.metrics         import mean_squared_error
from   sklearn.metrics         import mean_absolute_error 
from sklearn.pipeline        import Pipeline
#from   category_encoders       import *
from   sklearn.compose         import *
from   sklearn.experimental    import enable_iterative_imputer
from   sklearn.impute          import *
from   sklearn.preprocessing   import *


In [ ]:
# !pip install rfpimp

In [ ]:
data = pd.read_excel('RGG_RSS_final.xlsx')


array(['Recycle', 'Garbage/Compost', 'Garbage', 'Compost',
       'Garbage/Recycle'], dtype=object)

In [ ]:
data.loc[data['Neighborhood']]

,Commodity,Vehicle Type,Inside/Curb,16 gal,20 gal,32 gal,64 gal,96 gal,Meandor,Route,...,Day,#Units,Time,Company,Outlier,Truck,Neighborhood,Address_Street,Code,Has Key
0,Recycle,S-HEIL,C,0,0,1,2,2,NaN,912,...,2,5,118.0,RSS,0,SL,Inner Richmond,5128_geary,NaN,False
1,Recycle,S-HEIL,C,0,0,0,0,1,NaN,912,...,2,1,59.0,RSS,0,SL,Inner Richmond,5620_geary,NaN,False
2,Recycle,S-HEIL,I,0,0,0,0,1,NaN,912,...,2,1,86.0,RSS,0,SL,Inner Richmond,1947_clement,NaN,False
3,Recycle,S-HEIL,C,0,0,1,0,1,NaN,912,...,2,2,41.0,RSS,0,SL,Inner Richmond,1919_clement,NaN,False
4,Recycle,S-HEIL,C,0,0,1,0,0,NaN,912,...,2,1,31.0,RSS,0,SL,Inner Richmond,1909_clement,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,Compost,R-HEIL,I,0,0,0,1,0,NaN,302,...,4,1,98.0,RGG,0,RL,Chinatown,1220_jon,NaN,False
12822,Compost,R-HEIL,I,0,0,0,3,0,NaN,302,...,4,3,100.0,RGG,0,RL,Chinatown,1221_jon,NaN,False
12823,Compost,R-HEIL,C,0,0,0,2,0,NaN,302,...,4,2,66.0,RGG,0,RL,Chinatown,1000_california,NaN,False
12824,Compost,R-HEIL,IC,0,0,0,2,0,NaN,302,...,4,2,118.0,RGG,0,RL,Chinatown,250_eavenworth,NaN,False


In [ ]:
data['Commodity'].unique()

array(['Recycle', 'Garbage/Compost', 'Garbage', 'Compost',
       'Garbage/Recycle'], dtype=object)

In [ ]:
data['Day'].unique()

# data.loc[data['Day'].isnull()] 140 rows
# data['Day'].value_counts() --> 217 rows of 'na'

array([2, 5, 3, 1, 4, 'na', nan, 7], dtype=object)

In [ ]:
data['Day'] = data['Day'].astype(str)
data['Day'] = data['Day'].replace('nan','na')
data['Day'].unique()


array(['2', '5', '3', '1', '4', 'na', '7'], dtype=object)

In [ ]:
data['Company'].unique()

array(['RSS', 'RGG'], dtype=object)

In [ ]:
data['Truck'].unique()

array(['SL', nan, 'RL', 'FL'], dtype=object)

In [ ]:
data['Neighborhood'].unique()

array(['Inner Richmond', 'Outer Richmond', 'Excelsior',
       'Presidio Heights', 'Ocean View', 'West Twin Peaks',
       'Bernal Heights', 'Western Addition', 'Lakeshore',
       'Haight Ashbury', 'Visitation Valley', 'Inner Sunset',
       'Potrero Hill', 'Mission', 'Noe Valley', 'Outer Sunset',
       'Parkside', 'Downtown', 'Financial District', 'Pacific Heights',
       'SOMA', 'Presidio', 'Tresure Island', 'Russian Hill', 'Marina',
       'Chinatown', 'North Beach', 'Nob Hill'], dtype=object)

In [ ]:
data['Has Key'].unique()

array([False,  True])

In [ ]:

# X = data.loc[data['Outlier']==0].drop(columns='Time')

# y = data.loc[data['Outlier']==0]['Time']

# X = data.loc[data['Outlier']==0].drop(columns='Time')

# y = data.loc[data['Outlier']==0]['Time']


X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.2, random_state=16)

categorical_columns = ['Commodity', 'Day', 'Company', 'Truck', 'Neighborhood', 'Has Key']

numeric_columns = ['16 gal', '20 gal','32 gal', '64 gal', '96 gal','#Units']

categorical_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore'))])

numeric_pipe = Pipeline([('scaler', StandardScaler()),
                         ('imputer', SimpleImputer(strategy='median'))])

preprocessing = ColumnTransformer([('categorical', categorical_pipe, categorical_columns),
                                   ('numeric',  numeric_pipe, numeric_columns)])

transformer = QuantileTransformer(output_distribution='normal', n_quantiles=10000)

transformed_target_regr = TransformedTargetRegressor(regressor=LinearRegression(),
                                                    transformer=transformer)

pipe = Pipeline([('preprocessing', preprocessing), 
                 ('regressor',  transformed_target_regr)])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_validation)

mse = mean_squared_error(y_validation, y_pred)
print(f"mean squared error: {mse:,.2f}, Accuracy {100*pipe.score(X_validation, y_validation):.0f}%")



mean squared error: 2,606.37, Accuracy 47%


In [ ]:
# Try StandardScalar instead of Quintile Transformer --> no different than doing nothing to the target 

X = data.loc[data['Outlier']==0].drop(columns='Time')

y = data.loc[data['Outlier']==0]['Time']

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.2, random_state=16)

categorical_columns = ['Commodity', 'Day', 'Company', 'Truck', 'Neighborhood', 'Has Key']

numeric_columns = ['16 gal', '20 gal','32 gal', '64 gal', '96 gal','#Units']

categorical_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore'))])

numeric_pipe = Pipeline([('scaler', StandardScaler()),
                         ('imputer', SimpleImputer(strategy='median'))])

preprocessing = ColumnTransformer([('categorical', categorical_pipe, categorical_columns),
                                   ('numeric',  numeric_pipe, numeric_columns)])


transformer = StandardScaler()
transformed_target_regr = TransformedTargetRegressor(regressor=LinearRegression(),
                                                    transformer=transformer)

pipe = Pipeline([('preprocessing', preprocessing), 
                 ('regressor',  transformed_target_regr)])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_validation)

mae = mean_absolute_error(y_validation, y_pred)
print(f"mean absolute error: {mae:,.2f}, Accuracy {100*pipe.score(X_validation, y_validation):.0f}%")

# mean squared error: 2,607.26, Accuracy 58%

mean absolute error: 31.73, Accuracy 54%


In [ ]:
# Without quintile transformer 

X = data.drop(columns='Time')

y = data['Time']

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.2, random_state=16)

categorical_columns = ['Commodity', 'Day', 'Company', 'Neighborhood', 'Has Key']

numeric_columns = ['16 gal', '20 gal','32 gal', '64 gal', '96 gal','#Units']

categorical_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore'))])

numeric_pipe = Pipeline([('scaler', StandardScaler()),
                         ('imputer', SimpleImputer(strategy='median'))])

preprocessing = ColumnTransformer([('categorical', categorical_pipe, categorical_columns),
                                   ('numeric',  numeric_pipe, numeric_columns)])


pipe = Pipeline([('preprocessing', preprocessing), 
                 ('lr',  LinearRegression())])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_validation)

mae = mean_absolute_error(y_validation, y_pred)
print(f"mean absolute error: {mae:,.2f}, Accuracy {100*pipe.score(X_validation, y_validation):.0f}%")

# mean squared error: 2,607.26, Accuracy 58%
# mean absolute error: 32.46, Accuracy 58%

mean absolute error: 32.47, Accuracy 58%


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=938c6ad9-491d-4307-bf8a-c751a244ce4f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>